In [15]:
import numpy as np
import pandas as pd

def join_data(Order_Book, Kline_Data):
    """
    Joins Raw data into usable dataframes
    :param Order_Book: Raw Order Book Data (250ms ticks)
    :param Kline_Data: Raw Kline Data (250ms ticks)
    :return: Joined dataframe with no gaps
    """
    
    # Removing unwanted column
    Order_Book = Order_Book.drop("Unnamed: 0", axis=1)
    Kline_Data = Kline_Data.drop("Unnamed: 0", axis=1)
    
    # Removing NaN values from Order Book Data
    Order_Book[["BestBid", "BestAsk", "MidPrice"]] = Order_Book[["BestBid", "BestAsk", "MidPrice"]].fillna(method='ffill')
    
    # Adding Turnover to Kline Data
    Kline_Data["Turnover"] = Kline_Data["NumberOfTrades"]*Kline_Data["Price"]*10 #Tick Value = Contract size * Tick size = $100 * $0.1
    
    # Matching the timestamps of both datasets
    if Order_Book["Timestamp"][0] > Kline_Data["Timestamp"][0]:
        
        # We match the index with the minimum difference from the first timestamp of the later data
        index = Kline_Data.index[abs(Kline_Data["Timestamp"] - Order_Book["Timestamp"][0]) 
                                 == min(abs(Kline_Data["Timestamp"] - Order_Book["Timestamp"][0]))].to_list()[0]
        
        # Calculate the difference in time and drop the unnecessary rows
        diff = Kline_Data["Timestamp"][index] - Order_Book["Timestamp"][0]
        Kline_Data = Kline_Data.drop(Kline_Data.index[0:index]).reset_index(drop=True)
        
        # Match the times by substracting the difference
        Kline_Data["Timestamp"] = Kline_Data["Timestamp"] - diff
        
    else:
        # We match the index with the minimum difference from the first timestamp of the later data
        index = Order_Book.index[abs(Order_Book["Timestamp"] - Kline_Data["Timestamp"][0]) 
                                 == min(abs(Order_Book["Timestamp"] - Kline_Data["Timestamp"][0]))].to_list()[0]
        
        # Calculate the difference in time and drop the unnecessary rows
        diff = Order_Book["Timestamp"][index] - Kline_Data["Timestamp"][0]
        Order_Book = Order_Book.drop(Kline_Data.index[0:index]).reset_index(drop=True)
        
        # Match the times by substracting the difference
        Order_Book["Timestamp"] = Order_Book["Timestamp"] - diff
    
    # Removing duplicate rows just in case
    Order_Book[~Order_Book.duplicated('Timestamp', keep='first')]
    Kline_Data[~Kline_Data.duplicated('Timestamp', keep='first')]
    
    # Converting timestamps from unix to datetime
    Order_Book.index = pd.to_timedelta(Order_Book["Timestamp"].rename("Time"), "ms")
    Order_Book.drop("Timestamp", axis=1, inplace=True)
    Kline_Data.index = pd.to_timedelta(Kline_Data["Timestamp"].rename("Time"), "ms")
    Kline_Data.drop("Timestamp", axis=1, inplace=True)
    
    # Upsampling to 250ms data incase of gaps
    Order_Book = Order_Book.resample("250ms").last().ffill()
    Kline_Data = Kline_Data.resample("250ms").last().ffill()
    
    # Returns joined dataset
    return Order_Book.join(Kline_Data)

In [16]:
def linear_data(Order_Book, Kline_data, l=5, d=20):
    """
    Build up linear data for linear model
    :param Order_Book: Raw Order Book Data (250ms ticks)
    :param Kline_Data: Raw Kline Data (250ms ticks)
    :param l: the no. of LAGS for VOI and OIR determined by the ACF Plot
    :param d: the no. of DELAYS (in future) for calculating the mid price change
    :return: Dataframe with the required metrics
    """
    
    # Joining raw data into a single dataframe
    data = join_data(Order_Book, Kline_data)
    
    # Specify naming convention
    convention = {
              "BestBid":"BidDiff", 
              "BidVol":"BVolDiff",
              "BestAsk": "AskDiff",
              "AskVol": "AVolDiff",
              "Turnover": "TurnDiff",
              "Volume": "VolDiff"
                }
    
    # Calculating first diferrences of various columns and dropping null rows
    ldata = data[["BestBid", "BidVol", "BestAsk", "AskVol", "Turnover", "Volume"]].diff().rename(columns=convention)
    ldata[["BidVol", "AskVol", "MidPrice", "Price"]] = data[["BidVol", "AskVol", "MidPrice", "Price"]]
    ldata["Spread"] = data["BestAsk"] - data["BestBid"]
    ldata.drop(ldata.index[0], inplace=True)
    
    # Calculating Mid Price Change for given delays
    ldata["MPC"] = ldata["MidPrice"].shift(-1).rolling(d).mean().shift(1-d) - ldata["MidPrice"]
    
    # Calculating Mid Price Basis
    ldata["MPB"] = np.where(ldata["VolDiff"] != 0, ((ldata.iloc[:,4]/ldata.iloc[:,5])/300), np.nan)
    ldata["MPB"] = ldata["MPB"].fillna(method='ffill')
    index_mpb = ldata.columns.get_loc("MPB")
    index_mp = ldata.columns.get_loc("MidPrice")
    ldata.iloc[0, index_mpb] = ldata.iloc[0, index_mp]
    
    # Calculating OIR
    ldata["OIR_(t)"] = (ldata["BidVol"] - ldata["AskVol"])/(ldata["BidVol"] + ldata["AskVol"])
    
    # Calculating VOI
    dBid = pd.Series(np.where(ldata["BidDiff"] < 0, 0, 
                              np.where(ldata["BidDiff"] == 0, ldata["BVolDiff"], ldata["BidVol"])), index=ldata.index)
    dAsk = pd.Series(np.where(ldata["AskDiff"] < 0, ldata["AskVol"], 
                              np.where(ldata["AskDiff"] == 0, ldata["AVolDiff"], 0)), index=ldata.index)
    ldata["VOI_(t)"] = dBid - dAsk
    
    # Calculating VOI for each lag by shifting data
    for i in range(1, l+1):
        ldata[f"OIR_(t-{i})"] = ldata["OIR_(t)"].shift(i)
        ldata[f"VOI_(t-{i})"] = ldata["VOI_(t)"].shift(i)
    # Dropping irrelevant columns
    ldata = ldata.drop(columns=ldata.columns[:8])
    
    # Return dataframe with required metrics
    return ldata.dropna()

In [17]:
df1 = pd.read_csv("OrderBook_11.csv")
df2 = pd.read_csv("KlineData_11.csv")
linear_data(df1, df2)

,MidPrice,Price,Spread,MPC,MPB,OIR_(t),VOI_(t),OIR_(t-1),VOI_(t-1),OIR_(t-2),VOI_(t-2),OIR_(t-3),VOI_(t-3),OIR_(t-4),VOI_(t-4),OIR_(t-5),VOI_(t-5)
Time,,,,,,,,,,,,,,,,,
19458 days 20:21:57.347000,30050.40,30050.4,0.0,12.3000,1.298451e+04,1.000000,0.000,-1.000000,-5.167,0.088072,5.470,-0.370424,1.908,-0.453648,0.426,-0.470170,-0.020
19458 days 20:21:57.597000,30050.45,30050.5,0.1,12.3150,1.011063e+06,0.009325,1.996,1.000000,0.000,-1.000000,-5.167,0.088072,5.470,-0.370424,1.908,-0.453648,0.426
19458 days 20:21:57.847000,30096.65,30050.5,-92.3,-36.1275,7.627030e+03,-1.000000,-12.964,0.009325,1.996,1.000000,0.000,-1.000000,-5.167,0.088072,5.470,-0.370424,1.908
19458 days 20:21:58.097000,30050.45,30050.4,0.1,10.2050,2.231699e+04,0.460814,16.152,-1.000000,-12.964,0.009325,1.996,1.000000,0.000,-1.000000,-5.167,0.088072,5.470
19458 days 20:21:58.347000,30050.45,30050.4,0.1,10.3400,5.163299e+04,0.446795,-0.196,0.460814,16.152,-1.000000,-12.964,0.009325,1.996,1.000000,0.000,-1.000000,-5.167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19459 days 17:31:16.597000,30007.75,30007.8,0.1,2.3225,1.250325e+05,0.692759,0.003,-1.000000,-0.029,0.692420,4.325,0.502593,-0.113,0.505830,0.433,0.460552,-4.277
19459 days 17:31:16.847000,30008.40,30009.0,-1.2,1.7725,1.359787e+04,1.000000,71.593,0.692759,0.003,-1.000000,-0.029,0.692420,4.325,0.502593,-0.113,0.505830,0.433
19459 days 17:31:17.097000,30009.10,30009.1,0.0,1.2700,2.973977e+04,-1.000000,0.000,1.000000,71.593,0.692759,0.003,-1.000000,-0.029,0.692420,4.325,0.502593,-0.113


In [12]:
%%timeit
linear_data(df1, df2)

2.32 s ± 204 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
